In [ ]:
# 13.07.23 BG - Dateiname nach oben + Symbol Dazu 
# 11.07.23 BG - leeres Datum bei MULTI wird korrekt abgebildet 
# 11.07.23 BG - Mit Ausgabe Dateinamen
# 09.07.23 BG - Acount ID weg 
# 06.07.23 BG - bessere Namensgebung 
# 06.07.23 BG - underlyingSymbol ergänzt 
# 05.07.23 BG - levelOfDetail ergänzt um später bei der Auswertung nach BaseCurrency filtern zu können
# 04.07.23 BG - currency -> IBcurrency 
# 22.06.23 Bg - Ausgabe mit +-Zeichen 
# 21.06.23 BG - Version 1.1. - Feldnamen sidn ale OK 
# 18.06.23 BG - amount fehlte als Feld 
# 15.06.23 BG - Version 1.0 - Fertig, Tabelle wird erfolgreich gefüllt 
# 15.06.23 BG - Neu gemacht, da irgend was weg geworfen 
# 12.06.23 BG - erster Wurf
#
# 
# verwendeter Report: Statment of Funds - Kapitalfluss Report 
#
# Infos zu dem Report 
# https://ibkrguides.com/reportingreference/reportguide/statement%20of%20fundsfq.htm
#

#------------ Diese Datei wird eingelesen 

#dateiname = "Statements_LX_2023-01_02.xml"
#dateiname = "Statements.xml"
#dateiname = "GmbH-Statements.xml"
dateiname = "alex-statements.xml"

# --------------------------

import locale
from xml.dom.minidom import parse
from datetime import datetime

import pyodbc  # Datenbankanbindung an Access 

locale.setlocale(locale.LC_ALL, 'de_DE')
# locale.setlocale(locale.decimal_point, ',' )
#print (locale.localeconv())


def insertStatements (dbCursor, statement):
    
    global lastdate 
    
    #print ("-")
    #print ("Print statement --------- ")
    t = ";" # Definiere einen Trenner zum Lesen oder Excel 
    #t = " - " 

    transactionID = statement.getAttribute ("transactionID") 
    if ( transactionID != ""):
        transactionID = int(transactionID )
    else:
        transactionID = 0 

    # accountID = statement.getAttribute ("accountId")   
    accountID = "---"
    
    # Datum muss passend konvertiert werden
    settledate = statement.getAttribute ("settleDate") 
    date = statement.getAttribute ("date") 
    # 11.07.23 Hack, wegen fehlendem Datum 
    # print (f"Datum : {date} Settle: {settledate}") 
    if settledate == "MULTI":
        date = lastdate
    else:
        lastdate = date 

    formatting = "%Y-%m-%d"
    datum = datetime.strptime(date, formatting)

    amount = float(statement.getAttribute ("amount") )
    TradeID = statement.getAttribute ("tradeID")
    if ( TradeID == "" ):
         TradeID = 0 

    IBcurrency = statement.getAttribute ("currency") # 04.07.23 BG Neu 
    levelOfDetail = statement.getAttribute ("levelOfDetail") # 05.07.23 BG Neu 
 
    balance = float(statement.getAttribute ("balance") )
    balance = round ( balance , 2 )

    description = statement.getAttribute ("description") 
    activityCode = statement.getAttribute ("activityCode") 
    activityDescription = statement.getAttribute ("activityDescription") 
    # 06.07.23 BG underlyingSymbol
    symbol = statement.getAttribute ("symbol")
    underlyingSymbol = statement.getAttribute ("underlyingSymbol")

    #print ( "Transaktion:" , transactionID, " - Account ID",accountID,  " - Datum: " , datum , " - TradeID ", \
    #       TradeID, " - Balance: ", balance, " - Beschreibung " , description , "Code: ", activityCode , " - Desc: ", activityDescription) 

    # In die Datenbank eintragen 
    
    #activityCode  activityDescription
    sql = "INSERT INTO statements (transactionID, accountID, datum , TradeID , amount, IBcurrency, levelOfDetail, balance , " 
    sql += "description , activityCode, activityDescription, symbol, underlyingSymbol) "
    sql += " VALUES (?, ?, ?, ?, ? , ? , ? , ? , ? , ?, ?, ?, ?) "
    data = (transactionID, accountID, datum , TradeID , amount, IBcurrency, levelOfDetail, balance , description, activityCode , activityDescription, symbol, underlyingSymbol)
    
    dbCursor.execute(sql, data)
    # dbConn.commit()


# --------- Start des Programms --------------------------


lastdate = None 

#Datenbank öffnen 
# Verbindung zur Access-Datenbank herstellen
DBdatenbank = ".\ib.accdb"
DBdriver="{Microsoft Access Driver (*.mdb, *.accdb)}"
print ("Datenbank: ", DBdriver , " ", DBdatenbank )
#dbConn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=", datenbank, ";')
#dbConn = pyodbc.connect(r'Driver=', DBdriver,';DBQ=', DBdatenbank, ';')

# Das andere Wollte nicht 
dbConn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=.\ib.accdb;')

# Cursor-Objekt erstellen
dbCursor = dbConn.cursor()

#alle Daten löschen 
sql= "DELETE FROM statements"
dbCursor.execute (sql) 



domobject = parse( dateiname )
elementname = 'StatementOfFundsLine'

print (f"Start mit {dateiname}")

i = 0 ; 
limit = 100  
for statement in domobject.getElementsByTagName( elementname ):
    #Übertragen der Daten in die Datenbank 
    insertStatements ( dbCursor, statement )

    i = i+1
    if i % limit == 0  :
        print ("")
        print (i, end="")
    print("+", end="")
  
print ("")
print ( "Anzahl Datensätze eingetragen :" , i ) 
      
# Änderungen in der Datenbank speichern
dbConn.commit()
# Verbindung schließen
dbConn.close()

print (f"Fertig mit {dateiname}")
    
